In [212]:
import pandas as pd
import numpy as np
import datetime as dt
import bokeh.plotting as bpl
import bokeh.palettes as bpt
import colorcet as cc

In [213]:
bpl.output_notebook()

Loading BokehJS ...

Para ejecutar estos análisis primero se tiene que ejecutar el notebook del archivo "datos_historicos.ipynb", ese notebook permite descargar los archivos de datos con los que trabaja este notebook.

In [214]:
primera_fecha = "04/12/2020" #No hay datos antes del 04/12/2020 en las bases federales
ultima_fecha = "06/29/2020"

In [215]:
fechas = pd.date_range(start=primera_fecha, end=ultima_fecha)

Carga de datos históricos y actuales y conversión a DataFrame, para cada archivo se agrega una columna al dataframe que hace referencia a la fecha de la publicación del archivo, el país va a estar roto por problemas de encoding. En libre office se pueden abrir como utf8 pero python detecta un caracter que no puede decodificar y no abre los archivos. Se están forzando a 'latin1' pero entonces los acentos no son correctos.

In [216]:
fechas_csv = fechas.strftime("%y%m%d")+"COVID19MEXICO.csv"

In [217]:
frames = []
for fecha in fechas_csv:
    data = pd.read_csv('./datos_federales/' + fecha,encoding="latin1", low_memory=False)
    data.loc[:,"FECHA_ARCHIVO"] = dt.datetime.strptime(fecha[:6],"%y%m%d").date()
    frames.append(data)

In [218]:
len(frames)

79

In [219]:
all_data = pd.concat(frames,axis=0,ignore_index=True)

Conversión de fechas a formato de fecha

In [220]:
all_data["FECHA_ARCHIVO"] = pd.to_datetime(all_data["FECHA_ARCHIVO"],format="%Y-%m-%d")
all_data["FECHA_ACTUALIZACION"] = pd.to_datetime(all_data["FECHA_ACTUALIZACION"],format="%Y-%m-%d")
all_data["FECHA_INGRESO"] = pd.to_datetime(all_data["FECHA_INGRESO"],format="%Y-%m-%d")
all_data["FECHA_DEF"] = pd.to_datetime(all_data["FECHA_DEF"].replace({"9999-99-99":None}),format="%Y-%m-%d")
all_data["FECHA_SINTOMAS"] = pd.to_datetime(all_data["FECHA_SINTOMAS"],format="%Y-%m-%d")

In [221]:
all_data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'RESULTADO', 'MIGRANTE', 'PAIS_NACIONALIDAD',
       'PAIS_ORIGEN', 'UCI', 'FECHA_ARCHIVO'],
      dtype='object')

In [1504]:
len(all_data)

18692517

In [334]:
confirmados = all_data[all_data["RESULTADO"]==1]
difuntos = confirmados[confirmados["FECHA_DEF"].notna()]

In [335]:
len(confirmados)

6227690

In [1505]:
confirmados[:3]

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI,FECHA_ARCHIVO
0,2020-04-19,09e8dc,2,9,15,1,15,15,37.0,2,...,2,2,2,2,1,99,99,97,1,2020-04-12
1,2020-04-19,0efbaf,2,9,28,2,16,28,32.0,1,...,2,2,2,1,1,99,99,97,97,2020-04-12
4,2020-04-19,0cdb12,1,12,9,2,99,9,12.0,2,...,2,2,2,2,1,99,MÃ©xico,97,2,2020-04-12


In [337]:
por_fecha = confirmados.groupby(["ID_REGISTRO","FECHA_ARCHIVO"]).size().reset_index().pivot(index="FECHA_ARCHIVO",columns="ID_REGISTRO",values=0).sort_index().T

In [338]:
por_fecha[:3]

FECHA_ARCHIVO,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,...,2020-06-20,2020-06-21,2020-06-22,2020-06-23,2020-06-24,2020-06-25,2020-06-26,2020-06-27,2020-06-28,2020-06-29
ID_REGISTRO,,,,,,,,,,,,,,,,,,,,,
00000e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
000015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
000019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [339]:
fechas_validas = por_fecha.columns

In [340]:
colores = list(bpt.inferno(len(fechas_validas)))
colores.reverse()

In [341]:
conf_acum = por_fecha.sum().to_frame("Confirmados").reset_index()

In [342]:
conf_acum["diferencia"] = conf_acum["Confirmados"].diff()

In [343]:
conf_acum[:3]

,FECHA_ARCHIVO,Confirmados,diferencia
0,2020-04-12,4661.0,NaN
1,2020-04-13,5014.0,353.0
2,2020-04-14,5399.0,385.0


In [344]:
p = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")),title="Nacional, casos")

In [345]:
p.vbar(x=conf_acum["FECHA_ARCHIVO"],bottom=0,top=conf_acum["Confirmados"],width=pd.Timedelta("1 days")*0.8,line_color="#A8A8A8",color="#E0E0E0",legend_label="Casos reportados en las conferencias")

GlyphRenderer(id='233275', ...)

In [346]:
for i,fecha in enumerate(fechas_validas):
    conf_temp = confirmados[confirmados["FECHA_ARCHIVO"]==fecha]
    conf_sint = conf_temp.groupby("FECHA_SINTOMAS").size().cumsum().to_frame("Confirmados_fs").reset_index()
    if (np.mod(i,15)==0 or i == len(fechas_validas)-1):
        p.line(x=conf_sint["FECHA_SINTOMAS"],y=conf_sint["Confirmados_fs"],color=colores[i],legend_label="Casos por inicio de síntomas al " + fecha.strftime("%d/%m/%Y"))
    else:
        p.line(x=conf_sint["FECHA_SINTOMAS"],y=conf_sint["Confirmados_fs"],color=colores[i])

In [347]:
p.xaxis.axis_label = 'Fecha'
p.yaxis.axis_label = 'Casos confirmados'
p.legend.location = "top_left"

In [348]:
bpl.show(p)

In [349]:
pday = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(pd.Timestamp("04-01-2020")-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")),title="Nacional, casos diarios")

In [350]:
pday.vbar(x=conf_acum["FECHA_ARCHIVO"],bottom=0,top=conf_acum["diferencia"],width=pd.Timedelta("1 days")*0.8,line_color="#A8A8A8",color="#E0E0E0",legend_label="Casos nuevos diarios reportados en las conferencias")

GlyphRenderer(id='242488', ...)

In [351]:
for i,fecha in enumerate(fechas_validas):
    conf_temp = confirmados[confirmados["FECHA_ARCHIVO"]==fecha]
    conf_sint = conf_temp.groupby("FECHA_SINTOMAS").size().to_frame("Confirmados_fs").reset_index()
    if (np.mod(i,15)==0 or i == len(fechas_validas)-1):
        pday.line(x=conf_sint["FECHA_SINTOMAS"],y=conf_sint["Confirmados_fs"],color=colores[i],legend_label="Casos por inicio de síntomas al " + fecha.strftime("%d/%m/%Y"),line_alpha=0.3)
    else:
        pday.line(x=conf_sint["FECHA_SINTOMAS"],y=conf_sint["Confirmados_fs"],color=colores[i],line_alpha=0.3)

In [352]:
pday.xaxis.axis_label = 'Fecha'
pday.yaxis.axis_label = 'Casos nuevos reportados'
pday.legend.location = "top_left"

In [353]:
bpl.show(pday)

In [354]:
s = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,
               x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [355]:
for i,fecha in enumerate(fechas_validas):
    conf_temp = confirmados[confirmados["FECHA_ARCHIVO"]==fecha]
    conf_sint = conf_temp.groupby("FECHA_SINTOMAS").size().cumsum()
    porcentajes = (conf_sint - por_fecha.sum())
    porc_plot = porcentajes[porcentajes.notna()]
    if (np.mod(i,15)==0 or i == len(fechas_validas)-1):
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i],legend_label="al " + fecha.strftime("%d/%m/%Y"))
    else:
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i])

In [356]:
s.xaxis.axis_label = 'Fecha'
s.yaxis.axis_label = 'Diferencia en casos'
s.legend.location = "top_left"

In [357]:
bpl.show(s)

In [358]:
s = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [359]:
for i,fecha in enumerate(fechas_validas):
    conf_temp = confirmados[confirmados["FECHA_ARCHIVO"]==fecha]
    conf_sint = conf_temp.groupby("FECHA_SINTOMAS").size().cumsum()
    porcentajes = (conf_sint - por_fecha.sum())*100/(por_fecha.sum())
    porc_plot = porcentajes[porcentajes.notna()]
    if (np.mod(i,15)==0 or i == len(fechas_validas)-1):
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i],legend_label="al " + fecha.strftime("%d/%m/%Y"))
    else:
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i])

In [360]:
s.xaxis.axis_label = 'Fecha'
s.yaxis.axis_label = 'Diferencia porcentual'
s.legend.location = "top_right"

In [361]:
bpl.show(s)

In [362]:
df_fecha = []
for i,fecha in enumerate(fechas_validas):
    conf_temp = confirmados[confirmados["FECHA_ARCHIVO"]==fecha]
    conf_sint = conf_temp.groupby("FECHA_SINTOMAS").size().cumsum()
    conf_sint.name = fecha
    df_fecha.append(conf_sint)

In [363]:
valores = pd.concat(df_fecha,axis=1).T[fechas_validas[:-1]]

In [364]:
cambios = valores.pct_change()*100

In [365]:
fechas_casos = cambios.columns

In [366]:
coloresc = list(bpt.inferno(len(fechas_casos)))
coloresc.reverse()

In [367]:
s1 = bpl.figure(plot_width=1000,plot_height=500)

In [368]:
for i,fecha in enumerate(fechas_casos):
    t_data = cambios[fecha].to_frame("Cambio")
    t_data["Valores"] = valores[fecha]
    tf_data = t_data[t_data["Valores"].notna()].reset_index()
    tf_data["dias"] = (tf_data["index"]-fecha).dt.days
    if (np.mod(i,15)==0 or i == len(fechas_casos)-1):
        s1.line(x=tf_data["dias"],y=tf_data["Cambio"],color=coloresc[i],legend_label = fecha.strftime("%d/%m/%Y"))
    else:
        s1.line(x=tf_data["dias"],y=tf_data["Cambio"],color=coloresc[i])

In [369]:
s1.xaxis.axis_label = 'Días'
s1.yaxis.axis_label = 'Cambio porcentual diario'
s1.legend.location = "top_right"

In [370]:
bpl.show(s1)

In [371]:
por_fecha_dif = difuntos.groupby(["ID_REGISTRO","FECHA_ARCHIVO"]).size().reset_index().pivot(index="FECHA_ARCHIVO",columns="ID_REGISTRO",values=0).sort_index().T

In [372]:
fechas_validas = por_fecha_dif.columns

In [373]:
colores = list(bpt.inferno(len(fechas_validas)))
colores.reverse()

In [374]:
dif_acum = por_fecha_dif.sum().to_frame("Decesos").reset_index()

In [375]:
dif_acum["diferencia"] = dif_acum["Decesos"].diff()

In [376]:
dif_acum[:3]

,FECHA_ARCHIVO,Decesos,diferencia
0,2020-04-12,296.0,NaN
1,2020-04-13,332.0,36.0
2,2020-04-14,406.0,74.0


In [377]:
p = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")),title="Nacional, defunciones")

In [378]:
p.vbar(x=dif_acum["FECHA_ARCHIVO"],bottom=0,top=dif_acum["Decesos"],width=pd.Timedelta("1 days")*0.8,line_color="#A8A8A8",color="#E0E0E0",legend_label="Decesos reportados en las conferencias")

GlyphRenderer(id='280802', ...)

In [379]:
for i,fecha in enumerate(fechas_validas):
    dif_temp = difuntos[difuntos["FECHA_ARCHIVO"]==fecha]
    dif_sint = dif_temp.groupby("FECHA_DEF").size().cumsum().to_frame("Difuntos_fs").reset_index()
    if (np.mod(i,15)==0 or i == len(fechas_validas)-1):
        p.line(x=dif_sint["FECHA_DEF"],y=dif_sint["Difuntos_fs"],color=colores[i],legend_label="Defunciones por inicio de síntomas al " + fecha.strftime("%d/%m/%Y"))
    else:
        p.line(x=dif_sint["FECHA_DEF"],y=dif_sint["Difuntos_fs"],color=colores[i])

In [380]:
p.xaxis.axis_label = 'Fecha'
p.yaxis.axis_label = 'Decesos confirmados'
p.legend.location = "top_left"

In [381]:
bpl.show(p)

In [382]:
pday = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(pd.Timestamp("04-01-2020")-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")),title="Nacional, defunciones diarias")

In [383]:
pday.vbar(x=dif_acum["FECHA_ARCHIVO"],bottom=0,top=dif_acum["diferencia"],width=pd.Timedelta("1 days")*0.8,line_color="#A8A8A8",color="#E0E0E0",legend_label="Decesos nuevos diarios reportados en las conferencias")

GlyphRenderer(id='290888', ...)

In [384]:
for i,fecha in enumerate(fechas_validas):
    dif_temp = difuntos[difuntos["FECHA_ARCHIVO"]==fecha]
    dif_sint = dif_temp.groupby("FECHA_DEF").size().to_frame("Difuntos_fs").reset_index()
    if (np.mod(i,15)==0 or i == len(fechas_validas)-1):
        pday.line(x=dif_sint["FECHA_DEF"],y=dif_sint["Difuntos_fs"],color=colores[i],legend_label="Decesos por inicio de síntomas al " + fecha.strftime("%d/%m/%Y"),line_alpha=0.3)
    else:
        pday.line(x=dif_sint["FECHA_DEF"],y=dif_sint["Difuntos_fs"],color=colores[i],line_alpha=0.3)

In [385]:
pday.xaxis.axis_label = 'Fecha'
pday.yaxis.axis_label = 'Decesos nuevos reportados'
pday.legend.location = "top_left"

In [386]:
bpl.show(pday)

In [389]:
s = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,
               x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [390]:
for i,fecha in enumerate(fechas_validas):
    dif_temp = difuntos[difuntos["FECHA_ARCHIVO"]==fecha]
    dif_sint = dif_temp.groupby("FECHA_DEF").size().cumsum()
    porcentajes = (dif_sint - por_fecha_dif.sum())
    porc_plot = porcentajes[porcentajes.notna()]
    if (np.mod(i,15)==0 or i == len(fechas_validas)-1):
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i],legend_label="al " + fecha.strftime("%d/%m/%Y"))
    else:
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i])

In [391]:
s.xaxis.axis_label = 'Fecha'
s.yaxis.axis_label = 'Diferencia en casos'
s.legend.location = "top_left"

In [392]:
bpl.show(s)

In [393]:
s = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [394]:
for i,fecha in enumerate(fechas_validas):
    dif_temp = difuntos[difuntos["FECHA_ARCHIVO"]==fecha]
    dif_sint = dif_temp.groupby("FECHA_DEF").size().cumsum()
    porcentajes = (dif_sint - por_fecha_dif.sum())*100/(por_fecha_dif.sum())
    porc_plot = porcentajes[porcentajes.notna()]
    if (np.mod(i,15)==0 or i == len(fechas_validas)-1):
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i],legend_label="al " + fecha.strftime("%d/%m/%Y"))
    else:
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i])

In [395]:
s.xaxis.axis_label = 'Fecha'
s.yaxis.axis_label = 'Diferencia porcentual'
s.legend.location = "top_right"

In [396]:
bpl.show(s)

In [397]:
df_fecha = []
for i,fecha in enumerate(fechas_validas):
    dif_temp = difuntos[difuntos["FECHA_ARCHIVO"]==fecha]
    dif_sint = dif_temp.groupby("FECHA_DEF").size().cumsum()
    dif_sint.name = fecha
    df_fecha.append(dif_sint)

In [398]:
valores = pd.concat(df_fecha,axis=1).T[fechas_validas[:-4]]

In [399]:
cambios = valores.pct_change()*100

In [400]:
fechas_casos = cambios.columns

In [401]:
coloresc = list(bpt.inferno(len(fechas_casos)))
coloresc.reverse()

In [402]:
s1 = bpl.figure(plot_width=1000,plot_height=500)

In [403]:
for i,fecha in enumerate(fechas_casos):
    t_data = cambios[fecha].to_frame("Cambio")
    t_data["Valores"] = valores[fecha]
    tf_data = t_data[t_data["Valores"].notna()].reset_index()
    tf_data["dias"] = (tf_data["index"]-fecha).dt.days
    if (np.mod(i,15)==0 or i == len(fechas_casos)-1):
        s1.line(x=tf_data["dias"],y=tf_data["Cambio"],color=coloresc[i],legend_label = fecha.strftime("%d/%m/%Y"))
    else:
        s1.line(x=tf_data["dias"],y=tf_data["Cambio"],color=coloresc[i])

In [404]:
s1.xaxis.axis_label = 'Días'
s1.yaxis.axis_label = 'Cambio porcentual diario'
s1.legend.location = "top_right"

In [405]:
bpl.show(s1)

In [406]:
all_data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'RESULTADO', 'MIGRANTE', 'PAIS_NACIONALIDAD',
       'PAIS_ORIGEN', 'UCI', 'FECHA_ARCHIVO'],
      dtype='object')

In [407]:
pacientes = all_data.groupby(["ID_REGISTRO","ENTIDAD_UM","ENTIDAD_RES","MUNICIPIO_RES","EDAD","EMBARAZO","HABLA_LENGUA_INDIG","DIABETES","EPOC","ASMA","INMUSUPR","HIPERTENSION","OTRA_COM","CARDIOVASCULAR","OBESIDAD","RENAL_CRONICA","TABAQUISMO","MIGRANTE"]).size().reset_index().drop(columns=[0])

In [408]:
varios = pacientes["ID_REGISTRO"].value_counts()[pacientes["ID_REGISTRO"].value_counts()>1].index

In [409]:
varios

Index(['113be4', '06609a', '0a2098', '06d1a0', '1df826', '1bbda1', '07e0e7',
       '15d77f', '12e6df', '1c7f44',
       ...
       '03aef3', '036a51', '0f2b20', '16bdae', '11b282', '1aa892', '1931b8',
       '1db096', '07b29e', '06e347'],
      dtype='object', length=2591)

In [410]:
len(varios)

2591

In [411]:
conf_validos = pd.merge(pacientes,por_fecha.reset_index()[~por_fecha.reset_index()["ID_REGISTRO"].isin(varios)],how="right",on="ID_REGISTRO")

In [1471]:
#conf_filtrados = conf_validos
conf_filtrados = conf_validos[(conf_validos["ENTIDAD_RES"]==32)]
#conf_filtrados = conf_validos[(conf_validos["ENTIDAD_RES"]==2)&(conf_validos["MUNICIPIO_RES"]==4)]
etiqueta = "Zacatecas"

In [1472]:
conf_filtrados[:3]

,ID_REGISTRO,ENTIDAD_UM,ENTIDAD_RES,MUNICIPIO_RES,EDAD,EMBARAZO,HABLA_LENGUA_INDIG,DIABETES,EPOC,ASMA,...,2020-06-20 00:00:00,2020-06-21 00:00:00,2020-06-22 00:00:00,2020-06-23 00:00:00,2020-06-24 00:00:00,2020-06-25 00:00:00,2020-06-26 00:00:00,2020-06-27 00:00:00,2020-06-28 00:00:00,2020-06-29 00:00:00
53,0001de,32,32,10.0,66,97,2,1,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
463,001001,32,32,17.0,26,97,2,2,2,2,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1353,002f66,32,32,10.0,28,97,2,2,2,2,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [1473]:
conf_fil_whole = confirmados[confirmados["ID_REGISTRO"].isin(conf_filtrados["ID_REGISTRO"].unique())]

In [1474]:
difuntos_fil = conf_fil_whole[conf_fil_whole["FECHA_DEF"].notna()]

In [1475]:
conf_acum = conf_filtrados[fechas_validas].sum().to_frame("Confirmados").reset_index()

In [1476]:
conf_acum["diferencia"] = conf_acum["Confirmados"].diff()

In [1477]:
conf_acum[:3]

,index,Confirmados,diferencia
0,2020-04-12,16.0,NaN
1,2020-04-13,17.0,1.0
2,2020-04-14,18.0,1.0


In [1478]:
p = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")),title=etiqueta + ", casos")

In [1479]:
p.vbar(x=conf_acum["index"],bottom=0,top=conf_acum["Confirmados"],width=pd.Timedelta("1 days")*0.8,line_color="#A8A8A8",color="#E0E0E0",legend_label="Casos reportados en las conferencias")

GlyphRenderer(id='3176728', ...)

In [1480]:
for i,fecha in enumerate(fechas_validas):
    conf_temp = conf_fil_whole[conf_fil_whole["FECHA_ARCHIVO"]==fecha]
    conf_sint = conf_temp.groupby("FECHA_SINTOMAS").size().cumsum().to_frame("Confirmados_fs").reset_index()
    if (np.mod(i,15)==0 or i == len(fechas_validas)-1):
        p.line(x=conf_sint["FECHA_SINTOMAS"],y=conf_sint["Confirmados_fs"],color=colores[i],legend_label="Casos por inicio de síntomas al " + fecha.strftime("%d/%m/%Y"))
    else:
        p.line(x=conf_sint["FECHA_SINTOMAS"],y=conf_sint["Confirmados_fs"],color=colores[i])

In [1481]:
p.xaxis.axis_label = 'Fecha'
p.yaxis.axis_label = 'Casos confirmados'
p.legend.location = "top_left"

In [1482]:
bpl.show(p)

In [1483]:
s = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(fechas_validas[0]-pd.Timedelta("15 days"),fechas_validas[-1]+pd.Timedelta("1 days")),title = etiqueta + ", casos diarios")

In [1484]:
s.vbar(x=conf_acum["index"],bottom=0,top=conf_acum["diferencia"],width=pd.Timedelta("1 days")*0.8,line_color="#A8A8A8",color="#E0E0E0",legend_label="Casos reportados en las conferencias")

GlyphRenderer(id='3210417', ...)

In [1485]:
for i,fecha in enumerate(fechas_validas):
    conf_temp = conf_fil_whole[conf_fil_whole["FECHA_ARCHIVO"]==fecha]
    conf_sint = conf_temp.groupby("FECHA_SINTOMAS").size().to_frame("Confirmados_fs").reset_index()
    if (np.mod(i,15)==0 or i == len(fechas_validas)-1):
        s.line(x=conf_sint["FECHA_SINTOMAS"],y=conf_sint["Confirmados_fs"],color=colores[i],legend_label="Casos por inicio de síntomas al " + fecha.strftime("%d/%m/%Y"),line_alpha=0.3)
    else:
        s.line(x=conf_sint["FECHA_SINTOMAS"],y=conf_sint["Confirmados_fs"],color=colores[i],line_alpha=0.3)

In [1486]:
s.xaxis.axis_label = 'Fecha'
s.yaxis.axis_label = 'Casos nuevos confirmados'
s.legend.location = "top_left"

In [1487]:
bpl.show(s)

In [1488]:
por_fecha_dif = difuntos_fil.groupby(["ID_REGISTRO","FECHA_ARCHIVO"]).size().reset_index().pivot(index="FECHA_ARCHIVO",columns="ID_REGISTRO",values=0).sort_index().T

In [1489]:
fechas_validas = por_fecha_dif.columns

In [1490]:
colores = list(bpt.inferno(len(fechas_validas)))
colores.reverse()

In [1491]:
dif_acum = por_fecha_dif.sum().to_frame("Decesos").reset_index()

In [1492]:
dif_acum["diferencia"] = dif_acum["Decesos"].diff()

In [1493]:
dif_acum[:3]

,FECHA_ARCHIVO,Decesos,diferencia
0,2020-04-12,2.0,NaN
1,2020-04-13,2.0,0.0
2,2020-04-14,2.0,0.0


In [1494]:
p = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")),title = etiqueta + ", defunciones")

In [1495]:
p.vbar(x=dif_acum["FECHA_ARCHIVO"],bottom=0,top=dif_acum["Decesos"],width=pd.Timedelta("1 days")*0.8,line_color="#A8A8A8",color="#E0E0E0",legend_label="Decesos reportados en las conferencias")

GlyphRenderer(id='3244284', ...)

In [1496]:
for i,fecha in enumerate(fechas_validas):
    dif_temp = difuntos_fil[difuntos_fil["FECHA_ARCHIVO"]==fecha]
    dif_sint = dif_temp.groupby("FECHA_DEF").size().cumsum().to_frame("Difuntos_fs").reset_index()
    if (np.mod(i,15)==0 or i == len(fechas_validas)-1):
        p.line(x=dif_sint["FECHA_DEF"],y=dif_sint["Difuntos_fs"],color=colores[i],legend_label="Defunciones por inicio de síntomas al " + fecha.strftime("%d/%m/%Y"))
    else:
        p.line(x=dif_sint["FECHA_DEF"],y=dif_sint["Difuntos_fs"],color=colores[i])

In [1497]:
p.xaxis.axis_label = 'Fecha'
p.yaxis.axis_label = 'Decesos confirmados'
p.legend.location = "top_left"

In [1498]:
bpl.show(p)

In [1499]:
pday = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(pd.Timestamp("04-01-2020")-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")),title = etiqueta + ", defunciones diarias")

In [1500]:
pday.vbar(x=dif_acum["FECHA_ARCHIVO"],bottom=0,top=dif_acum["diferencia"],width=pd.Timedelta("1 days")*0.8,line_color="#A8A8A8",color="#E0E0E0",legend_label="Decesos nuevos diarios reportados en las conferencias")

GlyphRenderer(id='3278329', ...)

In [1501]:
for i,fecha in enumerate(fechas_validas):
    dif_temp = difuntos_fil[difuntos_fil["FECHA_ARCHIVO"]==fecha]
    dif_sint = dif_temp.groupby("FECHA_DEF").size().to_frame("Difuntos_fs").reset_index()
    if (np.mod(i,15)==0 or i == len(fechas_validas)-1):
        pday.line(x=dif_sint["FECHA_DEF"],y=dif_sint["Difuntos_fs"],color=colores[i],legend_label="Decesos por inicio de síntomas al " + fecha.strftime("%d/%m/%Y"),line_alpha=0.3)
    else:
        pday.line(x=dif_sint["FECHA_DEF"],y=dif_sint["Difuntos_fs"],color=colores[i],line_alpha=0.3)

In [1502]:
pday.xaxis.axis_label = 'Fecha'
pday.yaxis.axis_label = 'Decesos nuevos reportados'
pday.legend.location = "top_left"

In [1503]:
bpl.show(pday)